# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [17]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [18]:
y = x + x

In [19]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{98751508235: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

98751508235

In [15]:
x.id

83355646363

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:94580715541 -> bob:58885208875]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [21]:
# try this project here!

In [22]:
alice = sy.VirtualWorker(hook,id='alice')

In [37]:
alice._objects

{}

In [38]:
bob._objects

{}

In [39]:
x = th.tensor([1,2,3,4,5])

In [40]:
x = x.send(bob,alice)

In [41]:
alice._objects

{27245025598: tensor([1, 2, 3, 4, 5])}

In [42]:
bob._objects

{7157539302: tensor([1, 2, 3, 4, 5])}

In [43]:
x = x.get()
x

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [45]:
print(bob._objects)
print(alice._objects)

{}
{}


# Lesson: Introducing Remote Arithmetic

In [27]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [28]:
x

(Wrapper)>[PointerTensor | me:7344279461 -> bob:74976771769]

In [29]:
y

(Wrapper)>[PointerTensor | me:68830170639 -> bob:70981483926]

In [30]:
z = x + y

In [31]:
z

(Wrapper)>[PointerTensor | me:79875157913 -> bob:79875157913]

In [32]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [33]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:28437210120 -> bob:28437210120]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [35]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [36]:
z = (x + y).sum()

In [37]:
z.backward()

In [38]:
x = x.get()

In [39]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [20]:
# try this project here!
import torch.nn as nn
import torch.optim as optim

# Lesson: Garbage Collection and Common Errors


In [44]:
bob = bob.clear_objects()

In [45]:
bob._objects

{}

In [28]:
alice = sy.VirtualWorker(hook,id='alice')

In [46]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{23831414651: tensor([1, 2, 3, 4, 5])}

In [48]:
del x

In [49]:
bob._objects

{}

In [50]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{92212512027: tensor([1, 2, 3, 4, 5])}

In [52]:
x = "asdf"

In [53]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [56]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [57]:
x = "asdf"

In [58]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [59]:
del x

In [60]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [61]:
bob = bob.clear_objects()
bob._objects

{}

In [62]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{17426510898: tensor([1, 2, 3, 4, 5])}

In [64]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [25]:
z = x + y

In [29]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [27]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:56870833357 -> bob:54017818479]
Tensor B: tensor([ 2,  4,  6,  8, 10])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [69]:
from torch import nn, optim

In [33]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [34]:
# A Toy Model
model = nn.Linear(2,1)

In [35]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [36]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.4214)
tensor(0.2567)
tensor(0.1905)
tensor(0.1447)
tensor(0.1103)
tensor(0.0841)
tensor(0.0641)
tensor(0.0489)
tensor(0.0374)
tensor(0.0285)
tensor(0.0218)
tensor(0.0166)
tensor(0.0127)
tensor(0.0097)
tensor(0.0074)
tensor(0.0057)
tensor(0.0043)
tensor(0.0033)
tensor(0.0025)
tensor(0.0019)


In [37]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [38]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [39]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [47]:
def train(iterations=7):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [48]:
train()

tensor(2.2921, requires_grad=True)
tensor(0.7054, requires_grad=True)
tensor(0.3253, requires_grad=True)
tensor(0.4856, requires_grad=True)
tensor(0.1678, requires_grad=True)
tensor(0.3020, requires_grad=True)
tensor(0.1007, requires_grad=True)
tensor(0.1882, requires_grad=True)
tensor(0.0618, requires_grad=True)
tensor(0.1185, requires_grad=True)
tensor(0.0386, requires_grad=True)
tensor(0.0755, requires_grad=True)
tensor(0.0245, requires_grad=True)
tensor(0.0486, requires_grad=True)


# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [49]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [50]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
x = x.send(alice)

In [52]:
bob._objects

{9537782664: tensor([1, 2, 3, 4, 5])}

In [53]:
alice._objects

{93638098862: (Wrapper)>[PointerTensor | alice:93638098862 -> bob:9537782664]}

In [54]:
y = x + x

In [55]:
y

(Wrapper)>[PointerTensor | me:11056744825 -> alice:78990863598]

In [56]:
bob._objects

{9537782664: tensor([1, 2, 3, 4, 5]),
 43387492424: tensor([ 2,  4,  6,  8, 10])}

In [58]:
alice._objects

{93638098862: (Wrapper)>[PointerTensor | alice:93638098862 -> bob:9537782664],
 78990863598: (Wrapper)>[PointerTensor | alice:78990863598 -> bob:43387492424]}

In [59]:
jon = sy.VirtualWorker(hook, id="jon")

In [60]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [61]:
bob._objects

{98935835467: tensor([1, 2, 3, 4, 5])}

In [62]:
alice._objects

{42459924887: (Wrapper)>[PointerTensor | alice:42459924887 -> bob:98935835467]}

In [63]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:42459924887 -> bob:98935835467]

In [64]:
bob._objects

{98935835467: tensor([1, 2, 3, 4, 5])}

In [65]:
alice._objects

{}

In [66]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [67]:
bob._objects

{}

In [68]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [69]:
bob._objects

{64546222989: tensor([1, 2, 3, 4, 5])}

In [70]:
alice._objects

{62305962813: (Wrapper)>[PointerTensor | alice:62305962813 -> bob:64546222989]}

In [71]:
del x

In [72]:
bob._objects

{}

In [73]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [74]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [75]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [76]:
bob._objects

{25129157822: tensor([1, 2, 3, 4, 5])}

In [77]:
alice._objects

{}

In [78]:
x.move(alice)

(Wrapper)>[PointerTensor | me:15580712811 -> alice:15580712811]

In [79]:
bob._objects

{}

In [80]:
alice._objects

{15580712811: tensor([1, 2, 3, 4, 5])}

In [81]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [82]:
bob._objects

{27924299605: tensor([1, 2, 3, 4, 5])}

In [83]:
alice._objects

{15580712811: tensor([1, 2, 3, 4, 5]),
 48261308079: (Wrapper)>[PointerTensor | alice:48261308079 -> bob:27924299605]}

In [84]:
x.remote_get()

(Wrapper)>[PointerTensor | me:33918765525 -> alice:48261308079]

In [85]:
bob._objects

{}

In [86]:
alice._objects

{15580712811: tensor([1, 2, 3, 4, 5]), 48261308079: tensor([1, 2, 3, 4, 5])}

In [87]:
x.move(bob)

(Wrapper)>[PointerTensor | me:33918765525 -> bob:33918765525]

In [88]:
x

(Wrapper)>[PointerTensor | me:33918765525 -> bob:33918765525]

In [89]:
bob._objects

{33918765525: tensor([1, 2, 3, 4, 5])}

In [90]:
alice._objects

{15580712811: tensor([1, 2, 3, 4, 5])}